In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from google.colab.output import clear

Choose which model to use for tracking

In [ ]:
chooseModel = 'small'  # 'nano' or 'small'

# Download Test Set and prepare for Tracking



In [ ]:
!wget http://detrac-db.rit.albany.edu/Data/DETRAC-test-data.zip
clear()

In [ ]:
%cd /content/

/content


In [ ]:
!mkdir data

In [ ]:
path2TestSet = "/content/DETRAC-test-data.zip"

In [ ]:
!unzip -q $path2TestSet -d data/

###Download the processed ground truth from my repo

In [ ]:
!git clone https://github.com/munir-fati-haji/Thesis-Multiple-Object-Tracking.git myGidhubRepo

Cloning into 'myGidhubRepo'...
remote: Enumerating objects: 54691, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 54691 (delta 4), reused 42 (delta 4), pack-reused 54649
Receiving objects: 100% (54691/54691), 52.61 MiB | 37.70 MiB/s, done.
Resolving deltas: 100% (2722/2722), done.
Updating files: 100% (56354/56354), done.


# Tracking and saving results and video


## Install and import packages



In [ ]:
!pip install ultralytics
!pip install lap
!pip install motmetrics

clear()

import os
import shutil
import ultralytics
from ultralytics import YOLO
import motmetrics as mm
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm

ultralytics.checks()

Ultralytics YOLOv8.0.99 🚀 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (4 CPUs, 25.5 GB RAM, 31.8/166.8 GB disk)


## Track and save video and txt files

In [ ]:
testDir = '/content/data/Insight-MVT_Annotation_Test/'

In [ ]:
model = f"/content/myGidhubRepo/data/weights/{chooseModel}/best.pt"

In [ ]:
model = YOLO(model)

In [ ]:
!mkdir pred vids

In [ ]:
def saveVideo(pics_path, vid_dir):
  frame_size =(960, 540)
  output = cv2.VideoWriter(vid_dir, cv2.VideoWriter_fourcc('M','J','P','G'), 20, frame_size)

  for filename in tqdm(sorted(os.listdir(pics_path))):
      frame = cv2.imread(pics_path + filename)
      output.write(frame)
  output.release()
  cv2.destroyAllWindows()

In [ ]:
for foldername in tqdm(os.listdir(testDir)):
  folderPath = testDir + foldername + "/*.jpg"

  model.track(source=folderPath, tracker="bytetrack.yaml", save_txt=True,
              save=True, show_conf=False, line_thickness = 2)
  
  resultsPath = 'runs/detect/track/labels/'
  pics_path = 'runs/detect/track/'
  dest_dir = 'pred/'
  vid_dir = 'vids/' + foldername + ".avi"

  saveVideo(pics_path, vid_dir)
  shutil.move(resultsPath, os.path.join(dest_dir, foldername))
  shutil.rmtree('runs/')

Streaming output truncated to the last 5000 lines.
image 590/1030 /content/data/Insight-MVT_Annotation_Test/MVI_40711/img00590.jpg: 384x640 13 vehicles, 11.0ms
image 591/1030 /content/data/Insight-MVT_Annotation_Test/MVI_40711/img00591.jpg: 384x640 13 vehicles, 11.1ms
image 592/1030 /content/data/Insight-MVT_Annotation_Test/MVI_40711/img00592.jpg: 384x640 13 vehicles, 11.1ms
image 593/1030 /content/data/Insight-MVT_Annotation_Test/MVI_40711/img00593.jpg: 384x640 13 vehicles, 11.0ms
image 594/1030 /content/data/Insight-MVT_Annotation_Test/MVI_40711/img00594.jpg: 384x640 13 vehicles, 11.1ms
image 595/1030 /content/data/Insight-MVT_Annotation_Test/MVI_40711/img00595.jpg: 384x640 13 vehicles, 11.1ms
image 596/1030 /content/data/Insight-MVT_Annotation_Test/MVI_40711/img00596.jpg: 384x640 13 vehicles, 11.1ms
image 597/1030 /content/data/Insight-MVT_Annotation_Test/MVI_40711/img00597.jpg: 384x640 13 vehicles, 11.1ms
image 598/1030 /content/data/Insight-MVT_Annotation_Test/MVI_40711/img00598.j

# Evaluation of Tracking

In [ ]:
(width, height) = (960, 540)

In [ ]:
predDir = 'pred/'

In [ ]:
with open('summary.txt', 'w') as f:
  for foldername in tqdm(os.listdir(predDir)):
    # Create an accumulator that will be updated during each frame
    acc = mm.MOTAccumulator(auto_id=True)
    pred_directory = 'pred/' + foldername
    gt_directory = '/content/myGidhubRepo/data/gt/' + foldername

    pred = []
    gt = []
    

    for filename in os.listdir(pred_directory):
      if filename.endswith('.txt') and os.stat(os.path.join(gt_directory, filename)).st_size != 0 and os.stat(os.path.join(pred_directory, filename)).st_size != 0:
        if os.stat(os.path.join(pred_directory, filename)).st_size == 0:
          continue
        pred_path = os.path.join(pred_directory, filename)
        gt_path = os.path.join(gt_directory, filename)
        pred_df = pd.read_csv(pred_path, sep=' ', header=None)
        gt_df = pd.read_csv(gt_path, sep=' ', header=None)

        pred_df.iloc[:, 1:5] *= np.array([width, height, width, height])


        # Move last column of pred_df to the first position and remove original first column
        pred_df = pred_df.iloc[:, [-1] + list(range(1, pred_df.shape[1]-1))]

        pred.append(pred_df.to_numpy())
        gt.append(gt_df.to_numpy())
    for predictions, ground_truth in zip(pred, gt):
      # Convert ground truth and predictions to the required format
      gt_ids = [str(obj_id) for obj_id, x, y, width, height in ground_truth]
      pred_ids = [str(obj_id) for prediction in predictions if len(prediction) == 5 for obj_id, x, y, width, height in [prediction]]

      #pred_ids = [str(obj_id) for obj_id, x, y, width, height in predictions]

      gt_frame = np.array([[x, y, x+width, y+height] for obj_id, x, y, width, height in ground_truth], dtype=float)
      pred_frame = np.array([[x, y, x+width, y+height] for prediction in predictions if len(prediction) == 5 for obj_id, x, y, width, height in [prediction]], dtype=float)

      
      #pred_frame = np.array([[x, y, x+width, y+height] for obj_id, x, y, width, height in predictions], dtype=float)


      # Use the IoU (Intersection over Union) method for computing distances
      distance_matrix = mm.distances.iou_matrix(gt_frame, pred_frame, max_iou=1)
      #print(distance_matrix)

      # Update the accumulator
      acc.update(gt_ids, pred_ids, distance_matrix)
      # Compute metrics
    mh = mm.metrics.create()
    summary = mh.compute(acc, metrics=['num_frames', 'mota', 'motp', 
                                        'idp', 'idr','idf1', 'mostly_tracked',
                                      'partially_tracked', 'mostly_lost', 'precision', 
                                      'recall','num_switches'], name=foldername)
    # Convert the summary dictionary to a string
    nameMap = mm.io.motchallenge_metric_names
    nameMap['num_frames'] = 'NoF'
    strsummary = mm.io.render_summary(
        summary,
        formatters=mh.formatters,
        namemap=nameMap
    )
    # Convert the summary dictionary to a string
    f.write(strsummary)

100%|██████████| 40/40 [05:39<00:00,  8.49s/it]


In [ ]:
zipfile = f'{chooseModel}.zip'

In [ ]:
!zip -r $zipfile vids/ pred/ summary.txt

Streaming output truncated to the last 5000 lines.
  adding: pred/MVI_40742/img00983.txt (deflated 55%)
  adding: pred/MVI_40742/img00811.txt (deflated 53%)
  adding: pred/MVI_40742/img01554.txt (deflated 55%)
  adding: pred/MVI_40742/img00727.txt (deflated 54%)
  adding: pred/MVI_40742/img01176.txt (deflated 54%)
  adding: pred/MVI_40742/img00632.txt (deflated 56%)
  adding: pred/MVI_40742/img01426.txt (deflated 54%)
  adding: pred/MVI_40742/img00392.txt (deflated 55%)
  adding: pred/MVI_40742/img00071.txt (deflated 55%)
  adding: pred/MVI_40742/img00264.txt (deflated 56%)
  adding: pred/MVI_40742/img01479.txt (deflated 53%)
  adding: pred/MVI_40742/img01334.txt (deflated 54%)
  adding: pred/MVI_40742/img01236.txt (deflated 54%)
  adding: pred/MVI_40742/img00318.txt (deflated 55%)
  adding: pred/MVI_40742/img00222.txt (deflated 55%)
  adding: pred/MVI_40742/img00580.txt (deflated 55%)
  adding: pred/MVI_40742/img00925.txt (deflated 54%)
  adding: pred/MVI_40742/img00406.txt (deflated 

In [ ]:
!cp $zipfile /content/drive/MyDrive/
!cp summary.txt /content/drive/MyDrive/

In [ ]:
#!rm -rf /content/pred /content/vids /content/summary.txt